# Maze Router - run 1.0

### function to open file and extract data

In [ ]:
def parse_input_file(filename):
    grid_size = None
    obstacles = []
    nets = {}
    
    with open(filename, 'r') as f:
        lines = f.readlines()
        
    grid_size = lines[0].strip().split('x')
    width, height = int(grid_size[0]), int(grid_size[1])

    for line in lines[1:]:
        if line.startswith('OBS'):
            # Parse obstacle coordinates
            coords = line.strip()[5:-1].split(',')
            x, y = int(coords[0]), int(coords[1])
            obstacles.append((x, y))
        else:
            # Parse net definition
            parts = line.strip().split()
            net_name = parts[0]
            pins = []
            
            for pin_def in parts[1:]:
                pin_def = pin_def[1:-1]  
                layer, x, y = map(int, pin_def.split(','))
                pins.append((layer, x, y))
                
            nets[net_name] = pins
            
    return width, height, obstacles, nets


### creating the grid and putting 0s everywhere

In [ ]:
grid = []  
for layer_index in range(2):
    layer = []  
    for y in range(height):
        row = []  
        for x in range(width):
            row.append(0)  
        layer.append(row)
    grid.append(layer)

VIA_COST = 50  
WRONG_DIRECTION_COST = 20  

In [ ]:
# test 
line  = "NET1 (1,2,3) (1,4,5)"
parts = line.strip().split()
net_name = parts[0]
pins = []
ss = parts[2][1:-1]
layer, x, y = map(int, ss.split(','))
print(layer, x, y)

for pin_def in parts[1:]:
    pin_def = pin_def[1:-1]  # Remove parentheses
    layer, x, y = map(int, pin_def.split(','))
    pins.append((layer, x, y))



1 4 5


### Lee's Algorithm 

In [ ]:
def lee_algorithm(grid, source, target, via_cost, wrong_direction_cost):
    # Initialize
    queue = deque([source])
    visited = set([source])
    parent = {source: None}
    costs = {source: 0}
    
    source_layer, source_x, source_y = source
    target_layer, target_x, target_y = target
    # Wave propagation phase